In [19]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import os
import glob
import lasio
import dlisio as dlis

pd.options.display.max_columns = 10
pd.options.display.precision = 5
pd.options.display.max_colwidth = 30
pd.options.display.max_rows = 6

In [3]:
root_folder = r'C:\jupyter\volve_wells\Well_logs_pr_WELL'
las_files = []

for dirpath, dirnames, filenames in os.walk(root_folder):
    for filename in filenames:
        if filename.endswith('.las'):
            las_files.append(os.path.join(dirpath, filename))
        if filename.endswith('.LAS'):
            las_files.append(os.path.join(dirpath, filename))
        if filename.endswith('.dlis'):
            las_files.append(os.path.join(dirpath, filename))
        if filename.endswith('.DLIS'):
            las_files.append(os.path.join(dirpath, filename))

df_lst = []
for las_file in las_files:
    df_files = pd.DataFrame((las_file.split('\\')[4:])).T
    df_lst.append(df_files)
df_files = pd.concat(df_lst).reset_index(drop=True)
df_files.columns = ['well','folder','path_1', 'path_2']
df_files.sort_values(by='well', inplace=True)
df_files.folder.unique()

array(['05.PETROPHYSICAL INTERPRETATION', '06.LFP', '13.GEOCHEM',
       '04.COMPOSITE', '03.PRESSURE', '02.LWD_EWL', '01.MUD_LOG',
       '10.PRODUCTION LOGS', '11. INTEGRITY LOGS', '07.IMAGE',
       '08.VSP_VELOCITY', '12.BIOSTRAT'], dtype=object)

In [4]:
cpi_files = df_files[(df_files.folder == '05.PETROPHYSICAL INTERPRETATION') & 
                     (df_files.path_1 == 'CPI')].reset_index(drop=True)
interp_files = df_files[(df_files.folder == '05.PETROPHYSICAL INTERPRETATION') & 
                        (df_files.path_1 != 'CPI') & (df_files.path_2.isna())].reset_index(drop=True)

In [5]:
pp_files_wells = np.hstack((cpi_files.well.unique(), interp_files.well.unique()))

In [6]:
one_more_files = df_files[~df_files.well.isin(pp_files_wells)].reset_index(drop=True)

In [7]:
one_more_files.well.unique()

array(['15_9-F-7', '15_9-F-9', '15_9-F-9 A'], dtype=object)

In [8]:
one_more_files[one_more_files.path_1.str.contains('PETROPHYSICAL_COMPOSITE')]

,well,folder,path_1,path_2
1,15_9-F-7,04.COMPOSITE,WLC_PETROPHYSICAL_COMPOSIT...,NaN
9,15_9-F-9,04.COMPOSITE,WLC_PETROPHYSICAL_COMPOSIT...,NaN
22,15_9-F-9 A,04.COMPOSITE,WLC_PETROPHYSICAL_COMPOSIT...,NaN


In [9]:
las_data = []
for i in range(len(cpi_files)):
    full_path = os.path.join(root_folder, cpi_files.well[i], cpi_files.folder[i], cpi_files.path_1[i], cpi_files.path_2[i])
    las = lasio.read(full_path)
    df = las.df()
    df['WELL'] = cpi_files.well[i]
    las_data.append(df)
cpi_data = pd.concat(las_data).reset_index()
cpi_data = cpi_data[['WELL', 'DEPTH', 'BS', 'BVW', 'CARB_FLAG', 'COAL_FLAG', 'DELTA', 'GRMAX',
                    'GRMIN', 'J', 'KLHC_INT', 'KLOGH', 'M', 'N', 'PERF_FLAG', 'PHIF',
                    'PORD', 'RHOFL', 'RHOMA', 'RW', 'SAND_FLAG', 'SW', 'SWIRR', 'TEMP',
                    'VSH',  'KLOGV', 'BWV', 'DT']]
cpi_data

Only engine='normal' can read wrapped files


,WELL,DEPTH,BS,BVW,CARB_FLAG,...,TEMP,VSH,KLOGV,BWV,DT
0,15_9-19 A,3666.5916,8.5,0.11170,0.0,...,112.7249,0.59880,NaN,NaN,NaN
1,15_9-19 A,3666.7440,8.5,0.11224,0.0,...,112.7285,0.59955,NaN,NaN,NaN
2,15_9-19 A,3666.8964,8.5,0.11482,0.0,...,112.7321,0.60031,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
13180,15_9-19 S&SR,4618.0248,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,40.0
13181,15_9-19 S&SR,4618.1772,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,40.0
13182,15_9-19 S&SR,4618.3296,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,40.0


In [10]:
interp_files_input = []
interp_files_output = []
for idx, row in interp_files.iterrows():
    if 'INPUT' in row['path_1']:
        interp_files_input.append(interp_files.iloc[idx])
    if 'OUTPUT' in row['path_1']:
        interp_files_output.append(interp_files.iloc[idx])
interp_files_input = pd.DataFrame(interp_files_input).reset_index(drop=True)
interp_files_output = pd.DataFrame(interp_files_output).reset_index(drop=True)

In [11]:
las_data = []
las_file_err = []
for i in tqdm(range(len(interp_files_input))):
    full_path = os.path.join(root_folder, interp_files_input.well[i], interp_files_input.folder[i], interp_files_input.path_1[i])
    if 'las' in full_path or 'LAS' in full_path:
        las = lasio.read(full_path)
        df = las.df()
        df['WELL'] = interp_files_input.well[i]
        las_data.append(df)
    else:
        las_file_err.append(full_path)
interp_data_input = pd.concat(las_data).reset_index()
interp_data_input = interp_data_input[[ 'WELL', 'DEPTH', 'ABDCQF01', 'ABDCQF02', 'ABDCQF03', 'ABDCQF04', 'BS', 'CALI',
                                        'DRHO', 'DT', 'GR', 'NPHI', 'PEF', 'RACEHM', 'RACELM', 'RD', 'RHOB',
                                        'RM', 'ROP', 'RPCEHM', 'RPCELM', 'RT',  'DTS', 'NBGRCFM',
                                        'CARB_FLAG', 'COAL_FLAG', 'ROP5_RM', 'RS', 'SAND_FLAG']]

100%|██████████| 18/18 [00:04<00:00,  3.62it/s]


In [12]:
las_file_err

['C:\\jupyter\\volve_wells\\Well_logs_pr_WELL\\15_9-F-10\\05.PETROPHYSICAL INTERPRETATION\\WLC_PETRO_COMPUTED_INPUT_1.DLIS',
 'C:\\jupyter\\volve_wells\\Well_logs_pr_WELL\\15_9-F-14\\05.PETROPHYSICAL INTERPRETATION\\WLC_PETRO_COMPUTED_INPUT_1.DLIS',
 'C:\\jupyter\\volve_wells\\Well_logs_pr_WELL\\15_9-F-15\\05.PETROPHYSICAL INTERPRETATION\\WLC_PETRO_COMPUTED_INPUT_1.DLIS',
 'C:\\jupyter\\volve_wells\\Well_logs_pr_WELL\\15_9-F-15 A\\05.PETROPHYSICAL INTERPRETATION\\WLC_PETRO_COMPUTED_INPUT_1.DLIS',
 'C:\\jupyter\\volve_wells\\Well_logs_pr_WELL\\15_9-F-15 B\\05.PETROPHYSICAL INTERPRETATION\\WLC_PETRO_COMPUTED_INPUT_1.DLIS',
 'C:\\jupyter\\volve_wells\\Well_logs_pr_WELL\\15_9-F-15 C\\05.PETROPHYSICAL INTERPRETATION\\WLC_PETRO_COMPUTED_INPUT_1.DLIS',
 'C:\\jupyter\\volve_wells\\Well_logs_pr_WELL\\15_9-F-4\\05.PETROPHYSICAL INTERPRETATION\\WLC_PETRO_COMPUTED_INPUT_1.DLIS',
 'C:\\jupyter\\volve_wells\\Well_logs_pr_WELL\\15_9-F-5\\05.PETROPHYSICAL INTERPRETATION\\WLC_PETRO_COMPUTED_INPUT_1.DLI

In [13]:
len(interp_data_input.WELL.unique())

10

In [17]:
len(interp_files_input.well.unique())

18